In [29]:
import numpy as np
import pandas as pd


In [30]:
# Rread files
employees = pd.read_excel('EmployeeRoles.xlsx')
permissions = pd.read_excel('RolePermissions.xlsx')

In [31]:
# remove the admins and keep just the 'Name', 'Role', 'Permission', and 'Level' columns of the df
admins = employees[employees['Role'] == 'Administrator']['Name'].unique().tolist()
employoees = employees[~employees['Name'].isin(admins)][['Name', 'Role']]

In [33]:
# combine the employee file and the permissions file
all_permissions = employees.merge(permissions, how='outer', on='Role')

In [34]:
# some users have multiple roles, each with a differnt permissions level. we need to replace the pmermission levels with a binary (1=edit access or 0=no edit access); those values will be summed when we pivot the data; a user that has a permission that sums > 0 has access to that permission across any of their roles
df = all_permissions.replace({'Level': {'Full': 1, 'Edit': 1, 'Create': 1, 'View': 0, 'None': 0}})

# pivot the all_permissions df to get the permissions in the column headers; the index variable is all of the columns that we need to keep from the raw data
df = df.pivot_table(index='Name', values='Level', columns='Permission', aggfunc='sum').reset_index().rename_axis(None, axis=1).fillna(0)
for col in df.columns:
  if col != 'Name':
    df[col] = np.where(df[col] > 0, 'Yes', 'No')
df.head()

,Name,4599 GL Export Custom Field,4599 Transaction Child,AP Eligible Customer,AP Eligible Order,AP Minimum Purchase,AP Promotion Discount,AP Promotion Shipping Method,AP Promotion Shipping Price,AP Promotion Tier,...,Web Site Report,Web Store Report,Website (External) publisher,Work Calendar,Work Order,Work Order Close,Work Order Completion,Work Order Issue,Workflow,Zengin Bank Fee Schedule
0,Aaron Morgan,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
1,Abigail Howell,No,No,No,No,No,No,No,No,No,...,No,No,No,No,Yes,No,No,No,No,No
2,Adam Rose,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
3,Adrienne Potter,No,No,No,No,No,No,No,No,No,...,No,No,No,No,Yes,No,No,No,No,No
4,Alexandra Coleman,No,No,No,No,No,No,No,No,No,...,No,No,No,No,Yes,Yes,Yes,Yes,No,No


In [35]:
# pivot the all_permissions df with the role included for presentation purposes; end user will want to see which roles each user has and the level of their permissions
df2 = all_permissions.pivot_table(index=['Name','Role'], values='Level', columns='Permission', aggfunc=','.join).reset_index().rename_axis(None, axis=1).fillna(0).replace({0 : 'None'})
df2.head()

,Name,Role,4599 GL Export Custom Field,4599 Transaction Child,AP Eligible Customer,AP Eligible Order,AP Minimum Purchase,AP Promotion Discount,AP Promotion Shipping Method,AP Promotion Shipping Price,...,Web Site Report,Web Store Report,Website (External) publisher,Work Calendar,Work Order,Work Order Close,Work Order Completion,Work Order Issue,Workflow,Zengin Bank Fee Schedule
0,Aaron Morgan,Entity Employee Center,None,None,None,None,None,None,None,None,...,None,None,None,View,None,None,None,None,None,None
1,Aaron Morgan,Entity Production View,None,None,None,None,None,None,None,None,...,None,None,None,None,View,View,View,View,None,None
2,Abigail Howell,Company Manufacturing Technician,None,None,None,None,None,None,None,None,...,None,None,None,None,Create,View,View,View,None,None
3,Adam Rose,Company Employee Center,None,None,None,None,None,None,None,None,...,None,None,None,View,None,None,None,None,None,None
4,Adam Rose,Company Production View,None,None,None,None,None,None,None,None,...,None,None,None,None,View,View,View,View,None,None


In [36]:
# filter the df to those users that can create journal entries (Make Journal Entry) and approve journal entries (Journal Approval)
vf = pd.unique(df[(df['Make Journal Entry'] == "Yes") & (df['Journal Approval'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Make Journal Entry', 'Journal Approval']]
lf

,Name,Role,Make Journal Entry,Journal Approval
17,Alyssa Wilson,Entity Accounting - Maintenance and Invoicing,Edit,View
18,Alyssa Wilson,Entity Accounting Director Approval,None,None
19,Alyssa Wilson,Entity China Accountant,View,None
20,Alyssa Wilson,Entity Director of Shared Services,Edit,Full
21,Alyssa Wilson,Entity Fixed Assets Management,Full,None
...,...,...,...,...
762,Michelle Morton,Entity Sales Director,None,None
763,Michelle Morton,RF-SMART All Access - Example Only Do Not Use,None,None
917,Stephanie Moreno,Company Accounting - Dir of Shared Services - ...,Edit,Full
918,Stephanie Moreno,Custom EFT Role,View,None


In [37]:
# filter the df to those users users that can create customer invoices (Invoice) and can either receive customer payments (Customer Deposit) or record customer payments (Customer Payment)
vf = pd.unique(df[(df['Invoice'] == "Yes") & ((df['Customer Deposit'] == "Yes") | (df['Customer Payment'] == "Yes"))]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Invoice', 'Customer Deposit', 'Customer Payment']]
lf

,Name,Role,Invoice,Customer Deposit,Customer Payment
74,April Montgomery,Company Customer Service Rep - Europe,Edit,View,View
75,April Montgomery,Administrator,None,None,None
76,April Montgomery,Company - BIS Support Level 1,None,None,None
77,April Montgomery,Company Accounting - AP & Cash App - Europe,View,Edit,Edit
78,April Montgomery,Custom EFT Role,View,None,View
...,...,...,...,...,...
911,Stephanie Fowler,Company Accounting - AP & Cash App - Europe,View,Edit,Edit
912,Stephanie Fowler,Custom EFT Role,View,None,View
913,Stephanie Fowler,Company Accounting - Maint and Invoicing - Europe,Edit,View,View
914,Stephanie Fowler,Company Human Resources,None,None,None


In [38]:
# filter the df to those users that can create vendors (Vendors) and pay vendors (Pay Bills)
vf = pd.unique(df[(df['Vendors'] == "Yes") & (df['Pay Bills'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Customers', 'Customer Refund']]
lf

,Name,Role,Customers,Customer Refund
17,Alyssa Wilson,Entity Accounting - Maintenance and Invoicing,Edit,None
18,Alyssa Wilson,Entity Accounting Director Approval,None,None
19,Alyssa Wilson,Entity China Accountant,View,None
20,Alyssa Wilson,Entity Director of Shared Services,View,Edit
21,Alyssa Wilson,Entity Fixed Assets Management,View,None
...,...,...,...,...
919,Stephanie Moreno,Company Accounting - Maint and Invoicing - Europe,Edit,None
971,Valerie Adams,Entity Employee Center,None,None
972,Valerie Adams,Entity Accounting - Maintenance and Invoicing,Edit,None
973,Valerie Adams,Entity Warehouse Operations,View,None


In [39]:
# filter the df to those users that can create credit memos (Credit Memo) and can either receive customer payments (Customer Deposit) or record customer payments (Customer Payment)
vf = pd.unique(df[(df['Credit Memo'] == "Yes") & ((df['Customer Deposit'] == "Yes") | (df['Customer Payment'] == "Yes"))]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Credit Memo', 'Customer Deposit', 'Customer Payment']]
lf

,Name,Role,Credit Memo,Customer Deposit,Customer Payment
217,Daniel Silva,Entity Accounting - Maintenance and Invoicing,Edit,View,View
218,Daniel Silva,Entity Director of Shared Services,View,View,View
219,Daniel Silva,Entity Fixed Assets Management,None,None,None
220,Daniel Silva,Custom EFT Role,View,None,View
221,Daniel Silva,Entity Accounting - AP & Cash App,View,Edit,Edit
...,...,...,...,...,...
911,Stephanie Fowler,Company Accounting - AP & Cash App - Europe,View,Edit,Edit
912,Stephanie Fowler,Custom EFT Role,View,None,View
913,Stephanie Fowler,Company Accounting - Maint and Invoicing - Europe,Edit,View,View
914,Stephanie Fowler,Company Human Resources,None,None,None


In [40]:
# filter the df to those users that can create customers (Customers) and issue customer refunds (Customer Refund)
vf = pd.unique(df[(df['Customers'] == "Yes") & (df['Customer Refund'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Customers', 'Customer Refund']]
lf

,Name,Role,Customers,Customer Refund
17,Alyssa Wilson,Entity Accounting - Maintenance and Invoicing,Edit,None
18,Alyssa Wilson,Entity Accounting Director Approval,None,None
19,Alyssa Wilson,Entity China Accountant,View,None
20,Alyssa Wilson,Entity Director of Shared Services,View,Edit
21,Alyssa Wilson,Entity Fixed Assets Management,View,None
...,...,...,...,...
762,Michelle Morton,Entity Sales Director,Edit,None
763,Michelle Morton,RF-SMART All Access - Example Only Do Not Use,View,None
917,Stephanie Moreno,Company Accounting - Dir of Shared Services - ...,View,Edit
918,Stephanie Moreno,Custom EFT Role,View,View


In [41]:
# filter the df to those users that can create customers (Customers) and credit memos (Credit Memo)
vf = pd.unique(df[(df['Customers'] == "Yes") & (df['Credit Memo'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Customers', 'Credit Memo']]
lf

,Name,Role,Customers,Credit Memo
17,Alyssa Wilson,Entity Accounting - Maintenance and Invoicing,Edit,Edit
18,Alyssa Wilson,Entity Accounting Director Approval,None,None
19,Alyssa Wilson,Entity China Accountant,View,View
20,Alyssa Wilson,Entity Director of Shared Services,View,View
21,Alyssa Wilson,Entity Fixed Assets Management,View,None
...,...,...,...,...
919,Stephanie Moreno,Company Accounting - Maint and Invoicing - Europe,Edit,Edit
971,Valerie Adams,Entity Employee Center,None,None
972,Valerie Adams,Entity Accounting - Maintenance and Invoicing,Edit,Edit
973,Valerie Adams,Entity Warehouse Operations,View,None
